This project that I'm gonna do is "Seven Clean Steps To Reshape Your Data With Pandas Or How I Use Python Where Excel Fails" by Tich Mangono (https://towardsdatascience.com/seven-clean-steps-to-reshape-your-data-with-pandas-or-how-i-use-python-where-excel-fails-62061f86ef9c). 
Let's say that I'm gonna follow his concepts + steps then learn from it and try to explain with my language and knowledge. His article contains these concepts :
#### • multi-level indexing • pivoting • stacking • apply • lambda and list-comprehension
The dataset that used in this project is a spreadsheet with data on a public health intervention, consisting of many tabs, one tab per organization.
The problem was that the data was in wide format, but we needed a long format. So instead of copy-and-paste over and over, we can use Python’s Pandas library to automate this task !

#### Here’s a road map of what we will do with Pandas:
1. Set up the environment and load the data
2. Investigate the data
3. Parse the different data tabs
4. Standardize existing columns and create new ones
5. Clean up the data using “apply” and “lambda” functions
6. Reshape the data from wide to long by pivoting on multi-level indices and stacking
7. Concatenate and save the final results back to Excel

### So let's BEGIN !

# 1. Set up the environment and load the data

In [1]:
# just Lin-Bing and our data !
import pandas as pd
data = pd.ExcelFile('reshaping_data.xlsx')

# 2. Investigate the data

In [2]:
# let see all of its sheetname
data.sheet_names

['ABC_inc', 'HIJ_inc', 'OPQ_inc', 'XYZ_inc']

In [3]:
# and then its first 10 rows of first tab(if you use .head() it'll be 5 rows and you won't understand the sheet)
data.parse(sheet_name='ABC_inc', skiprows=0).head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Year1,...,NaN,NaN,Year4,NaN,NaN,NaN,NaN,Year5,NaN,NaN
6,NaN,district,province,partner,funding_source,NaN,NaN,2017,2017,2017,...,NaN,2020,2020,2020,2020,NaN,2021,2021,2021,2021
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-14yrs,15-29yrs,30+yrs,...,NaN,10-14yrs,15-29yrs,30+yrs,Total,NaN,10-14yrs,15-29yrs,30+yrs,Total
8,NaN,District 1,Region 1,partner 1,Souce 2,NaN,NaN,1296,383,1571,...,NaN,1906,1925,931,5465,NaN,61,353,1091,2409
9,NaN,District 2,Region 3,partner 6,Souce 5,NaN,NaN,722,232,1848,...,NaN,810,664,452,3665,NaN,989,374,1790,4320


Now you'll see that the header is start at row 6 !

# 3. Parse the different data tabs

Make a list of your target tab names. In our case we want all of them.
You can just make your own like just a ['ABC_inc', 'HIJ_inc'] if you want.

In [4]:
tabnames = data.sheet_names
# let's see what it look like
tabnames

['ABC_inc', 'HIJ_inc', 'OPQ_inc', 'XYZ_inc']

because the tabs have a same format so we will use only one them for this demonstration and we'll combine it at the end. Let parse the tab into a data frame, skipping the first 6 rows that we don't want. Don't forget to use “data.head()” to check result !

In [5]:
print(tabnames[0])

i = 0
df = data.parse(sheet_name=tabnames[i], skiprows=7)
df.head()

ABC_inc


,Unnamed: 0,district,province,partner,funding_source,Unnamed: 5,Unnamed: 6,2017,2017.1,2017.2,...,Unnamed: 21,2020,2020.1,2020.2,2020.3,Unnamed: 26,2021,2021.1,2021.2,2021.3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-14yrs,15-29yrs,30+yrs,...,NaN,10-14yrs,15-29yrs,30+yrs,Total,NaN,10-14yrs,15-29yrs,30+yrs,Total
1,NaN,District 1,Region 1,partner 1,Souce 2,NaN,NaN,1296,383,1571,...,NaN,1906,1925,931,5465,NaN,61,353,1091,2409
2,NaN,District 2,Region 3,partner 6,Souce 5,NaN,NaN,722,232,1848,...,NaN,810,664,452,3665,NaN,989,374,1790,4320
3,NaN,District 3,Region 1,partner 1,Souce 2,NaN,NaN,545,585,1736,...,NaN,1890,736,1414,5311,NaN,1215,112,1475,2824
4,NaN,District 4,Region 3,partner 6,Souce 5,NaN,NaN,631,1413,31,...,NaN,1646,960,209,2956,NaN,1392,936,701,4903


# 4. Standardize existing columns and create new ones

make a list of the header row and strip up to the 4th letter. This is the location and year information

In [6]:
cols1 = list(df.columns)
cols1_year = [str(x)[:4] for x in cols1]
cols1_year

['Unna',
 'dist',
 'prov',
 'part',
 'fund',
 'Unna',
 'Unna',
 '2017',
 '2017',
 '2017',
 '2017',
 'Unna',
 '2018',
 '2018',
 '2018',
 '2018',
 'Unna',
 '2019',
 '2019',
 '2019',
 '2019',
 'Unna',
 '2020',
 '2020',
 '2020',
 '2020',
 'Unna',
 '2021',
 '2021',
 '2021',
 '2021']

Now we got location and year information ! Now make another list of the first row, this is the age group information we need to preserve this information in the column name when we reshape the data .

In [7]:
cols2 = list(df.iloc[0,:])
cols2_age = [str(x) for x in cols2]
cols2_age

['nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total']

And that's a age-group information ! Then join the two lists to make a combined column name which preserves our location, year and age-group information.

In [8]:
cols = [x+"_"+y for x,y in zip(cols1_year,cols2_age)]
cols

['Unna_nan',
 'dist_nan',
 'prov_nan',
 'part_nan',
 'fund_nan',
 'Unna_nan',
 'Unna_nan',
 '2017_10-14yrs',
 '2017_15-29yrs',
 '2017_30+yrs',
 '2017_Total',
 'Unna_nan',
 '2018_10-14yrs',
 '2018_15-29yrs',
 '2018_30+yrs',
 '2018_Total',
 'Unna_nan',
 '2019_10-14yrs',
 '2019_15-29yrs',
 '2019_30+yrs',
 '2019_Total',
 'Unna_nan',
 '2020_10-14yrs',
 '2020_15-29yrs',
 '2020_30+yrs',
 '2020_Total',
 'Unna_nan',
 '2021_10-14yrs',
 '2021_15-29yrs',
 '2021_30+yrs',
 '2021_Total']

Nice ! Next Let's assign new column names to the dataframe

In [9]:
df.columns = cols
df.head(2)

,Unna_nan,dist_nan,prov_nan,part_nan,fund_nan,Unna_nan,Unna_nan,2017_10-14yrs,2017_15-29yrs,2017_30+yrs,...,Unna_nan,2020_10-14yrs,2020_15-29yrs,2020_30+yrs,2020_Total,Unna_nan,2021_10-14yrs,2021_15-29yrs,2021_30+yrs,2021_Total
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-14yrs,15-29yrs,30+yrs,...,NaN,10-14yrs,15-29yrs,30+yrs,Total,NaN,10-14yrs,15-29yrs,30+yrs,Total
1,NaN,District 1,Region 1,partner 1,Souce 2,NaN,NaN,1296,383,1571,...,NaN,1906,1925,931,5465,NaN,61,353,1091,2409


Then drop empty columns, Rename the useful columns.

In [10]:
df = df.drop(["Unna_nan"], axis=1).iloc[1:,:].rename(columns=    {'dist_nan':'district','prov_nan': 'province','part_nan':'partner','fund_nan':'financing_source'})
df.head(2)

,district,province,partner,financing_source,2017_10-14yrs,2017_15-29yrs,2017_30+yrs,2017_Total,2018_10-14yrs,2018_15-29yrs,...,2019_30+yrs,2019_Total,2020_10-14yrs,2020_15-29yrs,2020_30+yrs,2020_Total,2021_10-14yrs,2021_15-29yrs,2021_30+yrs,2021_Total
1,District 1,Region 1,partner 1,Souce 2,1296,383,1571,3250,189,854,...,491,2256,1906,1925,931,5465,61,353,1091,2409
2,District 2,Region 3,partner 6,Souce 5,722,232,1848,2802,972,69,...,245,2957,810,664,452,3665,989,374,1790,4320


In [11]:
# Engineer a new column for the organization, grab this name from the excel tab name
# This should read 'ABC inc' if executed correctly
df['main_organization'] = tabnames[i].split("_")[0] + " "+ tabnames[i].split("_")[1]
df.main_organization.head()

1    ABC inc
2    ABC inc
3    ABC inc
4    ABC inc
5    ABC inc
Name: main_organization, dtype: object

Let's take a break and look at our data frame

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 1 to 10
Data columns (total 25 columns):
district             10 non-null object
province             10 non-null object
partner              10 non-null object
financing_source     10 non-null object
2017_10-14yrs        10 non-null object
2017_15-29yrs        10 non-null object
2017_30+yrs          10 non-null object
2017_Total           10 non-null object
2018_10-14yrs        10 non-null object
2018_15-29yrs        10 non-null object
2018_30+yrs          10 non-null object
2018_Total           10 non-null object
2019_10-14yrs        10 non-null object
2019_15-29yrs        10 non-null object
2019_30+yrs          10 non-null object
2019_Total           10 non-null object
2020_10-14yrs        10 non-null object
2020_15-29yrs        10 non-null object
2020_30+yrs          10 non-null object
2020_Total           10 non-null object
2021_10-14yrs        10 non-null object
2021_15-29yrs        10 non-null object
2021_30+yrs      

Now we have 29 columns but all they're “object” data type, there should be a lot of numeric type and the column names still retain multiple levels of information, i.e. the year and the age-group !

# 5. Clean up the data using “apply” and “lambda” functions

Let’s remove more redundant columns and change the data types.

In [13]:
# Make lists of the columns which need attention and use this as reference to execute
to_remove = [c for c in df.columns if "Total" in c] # redundant
to_change = [c for c in df.columns if "yrs" in c] # numeric
# drop unwanted columns
# Notice that you need to specify inplace, otherwise pandas will return the data frame instead of changing it in place
df.drop(to_remove, axis=1, inplace= True) 
# Change the target column data types
for c in to_change:
    df[c] = df[c].apply(lambda x: pd.to_numeric(x))

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 1 to 10
Data columns (total 20 columns):
district             10 non-null object
province             10 non-null object
partner              10 non-null object
financing_source     10 non-null object
2017_10-14yrs        10 non-null int64
2017_15-29yrs        10 non-null int64
2017_30+yrs          10 non-null int64
2018_10-14yrs        10 non-null int64
2018_15-29yrs        10 non-null int64
2018_30+yrs          10 non-null int64
2019_10-14yrs        10 non-null int64
2019_15-29yrs        10 non-null int64
2019_30+yrs          10 non-null int64
2020_10-14yrs        10 non-null int64
2020_15-29yrs        10 non-null int64
2020_30+yrs          10 non-null int64
2021_10-14yrs        10 non-null int64
2021_15-29yrs        10 non-null int64
2021_30+yrs          10 non-null int64
main_organization    10 non-null object
dtypes: int64(15), object(5)
memory usage: 1.6+ KB


Now the Total column is gone and we got year column in numeric data type !

# 6. Reshape the data from wide to long by pivoting on multi-level indices and stacking

Right now the data is in a wide format, but we need to change it to long format so that we can easily transfer it to Excel where long formats lend themselves to quick pivot tables and dashboard creation very easily.

In [15]:
# First, select the columns to use for a multi-level index. This depends on your data
# Generally, you want all the identifier columns to be included in the multi-index 
# For this dataset, this is every non-numeric column
idx =['district','province','partner','financing_source','main_organization']
# Then pivot the dataset based on this multi-level index 
multi_indexed_df = df.set_index(idx)
multi_indexed_df.head(2)

,,,,,2017_10-14yrs,2017_15-29yrs,2017_30+yrs,2018_10-14yrs,2018_15-29yrs,2018_30+yrs,2019_10-14yrs,2019_15-29yrs,2019_30+yrs,2020_10-14yrs,2020_15-29yrs,2020_30+yrs,2021_10-14yrs,2021_15-29yrs,2021_30+yrs
district,province,partner,financing_source,main_organization,,,,,,,,,,,,,,,
District 1,Region 1,partner 1,Souce 2,ABC inc,1296,383,1571,189,854,339,840,773,491,1906,1925,931,61,353,1091
District 2,Region 3,partner 6,Souce 5,ABC inc,722,232,1848,972,69,1205,422,676,245,810,664,452,989,374,1790


After pivoting the data frame on our strategically engineered multi-level index, we will now stack all the numerical columns. This will give us the flexibility to reshape the data back to whatever level we want afterwards.

In [16]:
# Stack the columns to achieve the baseline long format for the data
stacked_df = multi_indexed_df.stack(dropna=False)
stacked_df.head(25) # check out the results!

district    province  partner    financing_source  main_organization               
District 1  Region 1  partner 1  Souce 2           ABC inc            2017_10-14yrs    1296
                                                                      2017_15-29yrs     383
                                                                      2017_30+yrs      1571
                                                                      2018_10-14yrs     189
                                                                      2018_15-29yrs     854
                                                                      2018_30+yrs       339
                                                                      2019_10-14yrs     840
                                                                      2019_15-29yrs     773
                                                                      2019_30+yrs       491
                                                                      2020_10-14yrs    1

It's gettin in shape !

In [24]:
# Now do a reset to disband the multi-level index, we only needed it to pivot our data during the reshape
long_df = stacked_df.reset_index()
long_df.head(3)

,district,province,partner,financing_source,main_organization,level_5,0
0,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_10-14yrs,1296
1,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_15-29yrs,383
2,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_30+yrs,1571


Notice that the “level_5” column contains two pieces of information. This was deliberately done from the beginning so as not to lose any information along the way as we drop some columns and rows. Now, lets use string manipulation to separate these and delete any redundancies after that.

In [18]:
# Make series of lists which split year from target age-group
# the .str attribute is how you manipulate the data frame objects and columns with strings in them
col_str = long_df.level_5.str.split("_") 
col_str.head(3)

# engineer the columns we want, one columns takes the first item in col_str and another columns takes the second 
long_df['target_year'] = [x[0] for x in col_str] 
long_df['target_age'] = [x[1] for x in col_str]
long_df['target_quantity'] = long_df[0] # rename this column
long_df.head(2)

,district,province,partner,financing_source,main_organization,level_5,0,target_year,target_age,target_quantity
0,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_10-14yrs,1296,2017,10-14yrs,1296
1,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_15-29yrs,383,2017,15-29yrs,383


In [25]:
# drop the now redundant columns
df_final = long_df.drop(['level_5', 0], axis=1)
df_final.head(2)

,district,province,partner,financing_source,main_organization
0,District 1,Region 1,partner 1,Souce 2,ABC inc
1,District 1,Region 1,partner 1,Souce 2,ABC inc


# 7. Concatenate and save the final results back to Excel

Now we have all the ingredients we need, we can define a function to automate the reshape, use iteration to apply this function to any number of tabs we want and then finally save this to our spreadsheet of choice! Notice that in the function I combine the shapes and add an assertion to check that the inputs are correct, but it mostly the same code.

In [20]:
# Now define a function for doing the reshape
def ReshapeFunc(excel_obj, i):
    """ Takes in an excel file object with multiple tabs in a wide format, and a specified index of the tab to be parsed and reshaped. Returns a data frame of the specified tab reshaped to long format"""

    tabnames = data.sheet_names
    assert i < len(tabnames), "Your tab index exceeds the number of available tabs, try a lower number" 
    
    # parse and clean columns
    df = excel_obj.parse(sheet_name=tabnames[i], skiprows=7)
    cols1 = [str(x)[:4] for x in list(df.columns)]
    cols2 = [str(x) for x in list(df.iloc[0,:])]
    cols = [x+"_"+y for x,y in zip(cols1,cols2)]
    df.columns = cols
    df = df.drop(["Unna_nan"], axis=1).iloc[1:,:].rename(columns={'dist_nan':'district','prov_nan': 'province','part_nan':'partner','fund_nan':'financing_source'})
    # new columns, drop some and change data type
    df['main_organization'] = tabnames[i].split("_")[0] + " "+ tabnames[i].split("_")[1]
    df.drop([c for c in df.columns if "Total" in c], axis=1, inplace= True) 
    
    for c in [c for c in df.columns if "yrs" in c]:
        df[c] = df[c].apply(lambda x: pd.to_numeric(x))
    # reshape - indexing, pivoting and stacking
    idx = ['district','province', 'partner','financing_source','main_organization']
    multi_indexed_df = df.set_index(idx)
    stacked_df = multi_indexed_df.stack(dropna=False)
    long_df = stacked_df.reset_index()
    
    # clean up and finalize
    col_str = long_df.level_5.str.split("_") 
    long_df['target_year'] = [x[0] for x in col_str] 
    long_df['target_age'] = [x[1] for x in col_str]
    long_df['target_quantity'] = long_df[0] # rename this column
    df_final = long_df.drop(['level_5', 0], axis=1)
    return df_final

In [21]:
# Check that our function works:
check_df = ReshapeFunc(data, 2)
check_df.head(2)

,district,province,partner,financing_source,main_organization,target_year,target_age,target_quantity
0,District 19,Region 4,partner 3,Souce 4,OPQ inc,2017,10-14yrs,1
1,District 19,Region 4,partner 3,Souce 4,OPQ inc,2017,15-29yrs,974


In [22]:
dfs_list = [ReshapeFunc(data, i) for i in range(4)]
concat_dfs  = pd.concat(dfs_list)
concat_dfs.to_excel("reshaping_result_long_format.xlsx")

In [23]:
concat_dfs

,district,province,partner,financing_source,main_organization,target_year,target_age,target_quantity
0,District 1,Region 1,partner 1,Souce 2,ABC inc,2017,10-14yrs,1296
1,District 1,Region 1,partner 1,Souce 2,ABC inc,2017,15-29yrs,383
2,District 1,Region 1,partner 1,Souce 2,ABC inc,2017,30+yrs,1571
3,District 1,Region 1,partner 1,Souce 2,ABC inc,2018,10-14yrs,189
4,District 1,Region 1,partner 1,Souce 2,ABC inc,2018,15-29yrs,854
5,District 1,Region 1,partner 1,Souce 2,ABC inc,2018,30+yrs,339
6,District 1,Region 1,partner 1,Souce 2,ABC inc,2019,10-14yrs,840
7,District 1,Region 1,partner 1,Souce 2,ABC inc,2019,15-29yrs,773
8,District 1,Region 1,partner 1,Souce 2,ABC inc,2019,30+yrs,491
9,District 1,Region 1,partner 1,Souce 2,ABC inc,2020,10-14yrs,1906


#### To be honest, It took me a lot of times to understand these codes and concepts. And at this moment I'm not sure that I'm truly understand these or not if I have to do project like this all by myself. So I'mma continue improving myself ! Thank you for you attention :)